# Part 4a - Model testing with model deployed
 

In [ ]:
import pandas as pd
df_test = pd.read_csv('data/test.csv')
ref_summaries = list(df_test['summary'])
texts = list(df_test['text'])

In [1]:
from sagemaker.huggingface import HuggingFaceModel

In [5]:
model_for_deployment = HuggingFaceModel(entry_point='inference.py',
                                        source_dir='inference_code',
                                        model_data='s3://sagemaker-us-east-1-905847418383/huggingface-pytorch-training-2021-12-01-09-15-45-087/output/model.tar.gz',
                                        role=role,
                                        pytorch_version='1.7.1',
                                        py_version='py36',
                                        transformers_version='4.6.1',
                                        )

In [6]:
predictor = model_for_deployment.deploy(initial_instance_count=1,
                                        instance_type='ml.g4dn.xlarge',
                                        serializer=sagemaker.serializers.JSONSerializer(),
                                        deserializer=sagemaker.deserializers.JSONDeserializer()
                                        )

-------!

In [10]:
data = {"inputs":texts[0], "parameters_list":[{"min_length": 5, "max_length": 20}]}
predictor.predict(data)

[['The blow-up of $\\mathbb{P}^3$ at 6 points']]

In [11]:
ref_summaries[0]

'Birational geometry of blow-ups of projective spaces along points and   lines'

In [12]:
candidate_summaries = []

for i, text in enumerate(texts):
    if i % 100 == 0:
        print(i)
    data = {"inputs":text, "parameters_list":[{"min_length": 5, "max_length": 20}]}
    candidate = predictor.predict(data)
    candidate_summaries.append(candidate[0][0])

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950


In [13]:
file = open("summaries/model-summaries.txt", "w")
for s in candidate_summaries:
    file.write(s + "\n")
file.close()

In [20]:
from datasets import load_metric
metric = load_metric("rouge")

In [21]:
def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

In [22]:
calc_rouge_scores(candidate_summaries, ref_summaries)

{'rouge1': 44.5, 'rouge2': 24.5, 'rougeL': 39.4, 'rougeLsum': 39.4}